In [3]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
all_data = []

def edit_affiliation(value):
    if "Age" in value:
        return np.nan
    else:
        return value

for i in range(1, 161):
    url = f"https://games.crossfit.com/leaderboard/quarterfinals/2023?quarterfinal=211&division=1&region=0&sort=0&page={i}"
    response = requests.get(url)

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(20)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'leaderboardSponsorVisible')))
    time.sleep(100)
    page_source = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_source, 'html.parser')
    leaderboard_visible_container = soup.find('div', id='leaderboardSponsorVisible')
    lb_main_container = leaderboard_visible_container.find('div', class_='lb-main')
    inner_containers = lb_main_container.find_all('div', class_='inner-container')
    inner_container = inner_containers[1]    
        
    table = inner_container.find_all('table', class_='desktop athletes')
    table_string = str(table)
    table_string = table_string.replace('[', '').replace(']', '')
    with open(f'leaderboard_html_tables/mens_qf_{i}.txt', 'w') as file:
        file.write(table_string)
   
    html_file_path = f'leaderboard_html_tables/mens_qf_{i}.txt'

    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    
    tbody = soup.find('tbody')
    rows = tbody.find_all('tr', class_='collapsed')
    
    data = []

    for row in rows:
        rank = row.find('td', class_='pos active-sort').find('div', class_='cell-inner').text.strip()
        full_name = row.find('td', class_='name').find('div', class_='full-name').text.strip()

        info_items = row.find('ul', class_='info').find_all('li')

        country = info_items[0].text.strip()
        region = info_items[1].text.strip()
        affiliation = info_items[2].text.strip()

        age = None
        height_weight = None

        for item in info_items:
            item_text = item.text.strip()
        
            if item_text.startswith('Age'):
                age = item_text
            elif '|' in item_text:
                height_weight = item_text
    
        data.append([rank, full_name, country, region, affiliation, age, height_weight])

    all_data.extend(data)

columns = ['Rank', 'Name', 'Country', 'Region', 'Affiliation', 'Age', 'Height_Weight']
df = pd.DataFrame(all_data, columns=columns)
df['Affiliation'] = df['Affiliation'].apply(edit_affiliation)
df['Age'] = df['Age'].str.replace("Age ","")
df[['Height', 'Weight']] = df['Height_Weight'].str.split('|', expand=True)
df['Height'] = df['Height'].str.strip()
df['Weight'] = df['Weight'].str.strip()
df = df.drop('Height_Weight', axis=1)
df.to_csv(f'scrapeddata/mens_quarterfinals_ranking_4.csv',index=False)
df

KeyboardInterrupt: 